In [62]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional

import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [3]:
# One-to-One Sequence Problems with Single Feature
# Each time step has one feature

In [4]:
# Create a dataset
X = list()
Y = list()
X = [x+1 for x in range(20)]
Y = [y * 15 for y in X]

print(X)
print(Y)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 165, 180, 195, 210, 225, 240, 255, 270, 285, 300]


In [5]:
# The input to LSTM layer should be in 3D shape i.e. (samples, time-steps, features).
# The samples are the number of samples in the input data. We have 20 samples in the input.
# The time-steps is the number of time-steps per sample. We can choose to have 1 time-step.
# Finally, features correspond to the number of features per time-step. We have 1 feature per time-step.
# Both input and output should have the same shape
# reshape() this 1D array into 3D array with 20 sample, 1 time steps and 1 feature at each time step.

In [6]:
X = array(X).reshape(20, 1, 1) 
Y = array(Y).reshape(20, 1, 1)
print(X.shape, Y.shape)

(20, 1, 1) (20, 1, 1)


In [7]:
#create a simple LSTM model with one LSTM layer
#input shape is (1,1) since our data has one time-step with one feature

In [8]:
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(1, 1))) #input layer
model.add(Dense(1)) #output layer
model.compile(optimizer='rmsprop', loss='mse') #optimizer & loss function
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10400     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# One Epoch is when the entire dataset is passed forward and backward through the neural network once
# Since one epoch is too big, its divided in several smaller batches
# As the number of epochs increases, more number of times the weight are changed in the neural network
# The curve goes from underfitting to optimal to overfitting curve.

In [18]:
#We train our model for 100 epochs with a batch size of 5.
#There are 20 samples. So for batch size of of 5, it will take 4 iterations to complete 1 epoch.
#After every epoch, the weights are changed and the curve goes from underfit->optimal->overfit

model.fit(X, Y, epochs=100,  batch_size=5)

Epoch 1/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0556
Epoch 2/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0459
Epoch 3/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0097
Epoch 4/100
4/4 [==============================] - 0s 2ms/step - loss: 0.0645
Epoch 5/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0610
Epoch 6/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0155
Epoch 7/100
4/4 [==============================] - 0s 1ms/step - loss: 0.1113
Epoch 8/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0207
Epoch 9/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0152
Epoch 10/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0644
Epoch 11/100
4/4 [==============================] - 0s 1ms/step - loss: 0.0499
Epoch 12/100
4/4 [==============================] - 0s 2ms/step - loss: 0.0093
Epoch 13/100
4/4 [==============================] - 0s 2ms/st

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 50)                10400     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 10,451
Trainable params: 10,451
Non-trainable params: 0
_________________________________________________________________


In [12]:
# To Do: predict the output for an input of 'x=10'
# The actual output should be 10 x 15 = 150.
# First, convert test data to the right shape i.e. 3D shape, as expected by LSTM. 
# The following script predicts the output for the number 30:

In [26]:
test_input = array([10])
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[149.67361]]


Solution via Stacked LSTM

In [28]:
model = Sequential()
model.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(1, 1))) #return sequence = True
model.add(LSTM(50, activation='relu')) # Second layer added
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
print(model.summary())



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 1, 50)             10400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 51        
Total params: 30,651
Trainable params: 30,651
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
history = model.fit(X, Y, epochs=1500,  batch_size=5) 

Epoch 1/1500
4/4 [==============================] - 0s 1ms/step - loss: 32252.7070
Epoch 2/1500
4/4 [==============================] - 0s 2ms/step - loss: 32226.4629
Epoch 3/1500
4/4 [==============================] - 0s 2ms/step - loss: 32202.4414
Epoch 4/1500
4/4 [==============================] - 0s 1ms/step - loss: 32170.6445
Epoch 5/1500
4/4 [==============================] - 0s 1ms/step - loss: 32136.0625
Epoch 6/1500
4/4 [==============================] - 0s 2ms/step - loss: 32088.7539
Epoch 7/1500
4/4 [==============================] - 0s 2ms/step - loss: 32028.1309
Epoch 8/1500
4/4 [==============================] - 0s 2ms/step - loss: 31948.3496
Epoch 9/1500
4/4 [==============================] - 0s 1ms/step - loss: 31846.0000
Epoch 10/1500
4/4 [==============================] - 0s 2ms/step - loss: 31686.5000
Epoch 11/1500
4/4 [==============================] - 0s 1ms/step - loss: 31482.2070
Epoch 12/1500
4/4 [==============================] - 0s 2ms/step - loss: 31219.9258
E

4/4 [==============================] - 0s 2ms/step - loss: 121.6386
Epoch 101/1500
4/4 [==============================] - 0s 2ms/step - loss: 120.4812
Epoch 102/1500
4/4 [==============================] - 0s 2ms/step - loss: 119.0243
Epoch 103/1500
4/4 [==============================] - 0s 2ms/step - loss: 117.7919
Epoch 104/1500
4/4 [==============================] - 0s 2ms/step - loss: 116.5541
Epoch 105/1500
4/4 [==============================] - 0s 2ms/step - loss: 115.3503
Epoch 106/1500
4/4 [==============================] - 0s 2ms/step - loss: 114.2772
Epoch 107/1500
4/4 [==============================] - 0s 2ms/step - loss: 112.8410
Epoch 108/1500
4/4 [==============================] - 0s 1ms/step - loss: 111.9256
Epoch 109/1500
4/4 [==============================] - 0s 2ms/step - loss: 110.3646
Epoch 110/1500
4/4 [==============================] - 0s 1ms/step - loss: 109.1695
Epoch 111/1500
4/4 [==============================] - 0s 1ms/step - loss: 107.9309
Epoch 112/1500
4/4 

4/4 [==============================] - 0s 2ms/step - loss: 33.1556
Epoch 200/1500
4/4 [==============================] - 0s 2ms/step - loss: 32.6473
Epoch 201/1500
4/4 [==============================] - 0s 2ms/step - loss: 32.2095
Epoch 202/1500
4/4 [==============================] - 0s 2ms/step - loss: 31.6296
Epoch 203/1500
4/4 [==============================] - 0s 1ms/step - loss: 31.1248
Epoch 204/1500
4/4 [==============================] - 0s 2ms/step - loss: 30.7090
Epoch 205/1500
4/4 [==============================] - 0s 2ms/step - loss: 30.0973
Epoch 206/1500
4/4 [==============================] - 0s 1ms/step - loss: 29.5809
Epoch 207/1500
4/4 [==============================] - 0s 2ms/step - loss: 29.1328
Epoch 208/1500
4/4 [==============================] - 0s 2ms/step - loss: 28.6716
Epoch 209/1500
4/4 [==============================] - 0s 2ms/step - loss: 28.2372
Epoch 210/1500
4/4 [==============================] - 0s 2ms/step - loss: 27.7324
Epoch 211/1500
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 5.3500
Epoch 300/1500
4/4 [==============================] - 0s 2ms/step - loss: 5.2210
Epoch 301/1500
4/4 [==============================] - 0s 2ms/step - loss: 5.1242
Epoch 302/1500
4/4 [==============================] - 0s 2ms/step - loss: 5.0269
Epoch 303/1500
4/4 [==============================] - 0s 1ms/step - loss: 4.9328
Epoch 304/1500
4/4 [==============================] - 0s 2ms/step - loss: 4.8277
Epoch 305/1500
4/4 [==============================] - 0s 2ms/step - loss: 4.7358
Epoch 306/1500
4/4 [==============================] - 0s 2ms/step - loss: 4.6504
Epoch 307/1500
4/4 [==============================] - 0s 2ms/step - loss: 4.6001
Epoch 308/1500
4/4 [==============================] - 0s 2ms/step - loss: 4.4708
Epoch 309/1500
4/4 [==============================] - 0s 2ms/step - loss: 4.3981
Epoch 310/1500
4/4 [==============================] - 0s 2ms/step - loss: 4.2993
Epoch 311/1500
4/4 [=======================

4/4 [==============================] - 0s 2ms/step - loss: 1.1386
Epoch 401/1500
4/4 [==============================] - 0s 2ms/step - loss: 1.1282
Epoch 402/1500
4/4 [==============================] - 0s 2ms/step - loss: 1.1178
Epoch 403/1500
4/4 [==============================] - 0s 2ms/step - loss: 1.1079
Epoch 404/1500
4/4 [==============================] - 0s 2ms/step - loss: 1.1113
Epoch 405/1500
4/4 [==============================] - 0s 2ms/step - loss: 1.0938
Epoch 406/1500
4/4 [==============================] - 0s 2ms/step - loss: 1.0851
Epoch 407/1500
4/4 [==============================] - 0s 2ms/step - loss: 1.0739
Epoch 408/1500
4/4 [==============================] - 0s 1ms/step - loss: 1.0655
Epoch 409/1500
4/4 [==============================] - 0s 2ms/step - loss: 1.0677
Epoch 410/1500
4/4 [==============================] - 0s 2ms/step - loss: 1.0476
Epoch 411/1500
4/4 [==============================] - 0s 2ms/step - loss: 1.0427
Epoch 412/1500
4/4 [=======================

...

Epoch 1006/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2896
Epoch 1007/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.2936
Epoch 1008/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2914
Epoch 1009/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2953
Epoch 1010/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2996
Epoch 1011/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2804
Epoch 1012/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.2886
Epoch 1013/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2971
Epoch 1014/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.3039
Epoch 1015/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2948
Epoch 1016/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2822
Epoch 1017/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2881
Epoch 1018/1500


4/4 [==============================] - 0s 2ms/step - loss: 0.2245
Epoch 1106/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2266
Epoch 1107/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2233
Epoch 1108/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2205
Epoch 1109/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2368
Epoch 1110/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.2207
Epoch 1111/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.2229
Epoch 1112/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2193
Epoch 1113/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2227
Epoch 1114/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2185
Epoch 1115/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2214
Epoch 1116/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.2190
Epoch 1117/1500
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.1675
Epoch 1206/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1705
Epoch 1207/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1687
Epoch 1208/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1705
Epoch 1209/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1667
Epoch 1210/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1688
Epoch 1211/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.1660
Epoch 1212/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1707
Epoch 1213/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1647
Epoch 1214/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1666
Epoch 1215/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1631
Epoch 1216/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.1662
Epoch 1217/1500
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.1221
Epoch 1306/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1276
Epoch 1307/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1294
Epoch 1308/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1270
Epoch 1309/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1257
Epoch 1310/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1199
Epoch 1311/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1233
Epoch 1312/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1224
Epoch 1313/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1196
Epoch 1314/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.1204
Epoch 1315/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1208
Epoch 1316/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.1268
Epoch 1317/1500
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.0962
Epoch 1406/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.0935
Epoch 1407/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.0941
Epoch 1408/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.0962
Epoch 1409/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.0980
Epoch 1410/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.0924
Epoch 1411/1500
4/4 [==============================] - 0s 1ms/step - loss: 0.0931
Epoch 1412/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.0917
Epoch 1413/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.0917
Epoch 1414/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.0923
Epoch 1415/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.0907
Epoch 1416/1500
4/4 [==============================] - 0s 2ms/step - loss: 0.0906
Epoch 1417/1500
4/4 [===========

In [30]:
test_input = array([10])
test_input = test_input.reshape((1, 1, 1))
test_output = model.predict(test_input, verbose=0)
print(test_output)

[[150.11133]]
